In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/embfts")
import pandas as pd
import matplotlib.pyplot as plt
from models.NonStationaryFtsPca import NonStationaryFtsPca
from pyFTS.benchmarks import Measures
from pyFTS.models.nonstationary import nsfts
import math
import datetime
import statistics
import numpy as np
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from util.DataSetUtil import DataSetUtil
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

### Dataset Energy Appliance Kaggle

In [2]:
data_set_util = DataSetUtil()

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/HomeC.csv', sep=',')
data = df.drop(labels=['time','House overall [kW]','cloudCover','summary','icon'], axis=1)
data = data.loc[0: : 10]
data.dropna(inplace=True)
data = data_set_util.clean_dataset(data)
data = data_set_util.series_to_supervised_miso(data, 1, 1, 'use [kW]')
data.head()


/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,humidity(t-1),visibility(t-1),apparentTemperature(t-1),pressure(t-1),windSpeed(t-1),windBearing(t-1),precipIntensity(t-1),dewPoint(t-1),precipProbability(t-1),use [kW](t)
10,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.585083
20,1.585083,0.003417,0.000050,0.022100,0.678733,0.440200,0.121450,0.007433,0.013583,0.000350,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,0.533817
30,0.533817,0.003450,0.000000,0.020633,0.062967,0.270033,0.004950,0.007033,0.013117,0.000750,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.190950
40,1.190950,0.003083,0.000217,0.387400,0.061883,0.256867,0.004933,0.101783,0.012533,0.000700,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.582083
50,1.582083,0.003067,0.000050,0.022050,0.681717,0.242433,0.005550,0.122400,0.013550,0.000567,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,0.837000


### Model: NonStationaryFtsPca - PCA-NSFTS

In [25]:
fts_pca = NonStationaryFtsPca(num_components_pca = 1,
                          fts_model = nsfts.NonStationaryFTS,
                          order_fts_model = 1,
                          npart = 60,
                          gamma = 0.5,
                          memory_window_error = 3)

### Sliding Window PyFTS: PCA-NSFTS

In [26]:
result = {
     "window": [],
     "rmse": [],
     "mape": [],
     "smape": [],
     "mae": [],
     "r2": []
}
steps_ahead = 1
tam = len(data)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
    if len(ttest) > 0:
        
        print('-' * 20)
        print(f'training window {(ct)}')
        ttrain = ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
        model_train, pca_train = fts_pca.run_train_pca_non_stationary(ttrain)
        y_test = ttest['use [kW](t-1)'].values
        forecast, data_test = fts_pca.run_test_target(y_test,steps_ahead)
        y_validation = ttest['use [kW](t)'].values
        
        y_validation = y_validation[:len(y_validation)-1]
        forecast = forecast[1:]
        
        #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        rmse = Measures.rmse(y_validation,forecast)
        mape = Measures.mape(y_validation,forecast)
        smape = Measures.smape(y_validation,forecast)
        mae = mean_absolute_error(y_validation, forecast)
        r2 = r2_score(y_validation, forecast)
        
        result["rmse"].append(rmse)
        result["mape"].append(mape)
        result["smape"].append(smape)
        result["mae"].append(mae)
        result["r2"].append(r2)
        result["window"].append(ct)
        
#         fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#         ax.plot(y_validation, label='Original')
#         ax.plot(forecast, label='Forecast')
#         handles, labels = ax.get_legend_handles_labels()
#         lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#         plt.show()
        
        
measures = pd.DataFrame(result)

--------------------
training window 0
--------------------
training window 1679
--------------------
training window 3358
--------------------
training window 5037
--------------------
training window 6716
--------------------
training window 8395
--------------------
training window 10074
--------------------
training window 11753
--------------------
training window 13432
--------------------
training window 15111
--------------------
training window 16790
--------------------
training window 18469
--------------------
training window 20148
--------------------
training window 21827
--------------------
training window 23506
--------------------
training window 25185
--------------------
training window 26864
--------------------
training window 28543
--------------------
training window 30222
--------------------
training window 31901
--------------------
training window 33580
--------------------
training window 35259
--------------------
training window 36938
--------------------

#### Sliding Windows Statistics: 

In [27]:
print("Sliding Windows Statistics (test): ")
measures

Sliding Windows Statistics (test): 


,window,rmse,mape,smape,mae,r2
0,0,0.372584,29.061222,12.044819,0.276097,0.730929
1,1679,0.128759,17.099635,8.052804,0.098787,0.963175
2,3358,0.290188,70.589376,23.347028,0.244603,0.565820
3,5037,0.257125,18.502948,8.609438,0.185091,0.880848
4,6716,0.144877,44.451343,11.013098,0.120932,0.937073
5,8395,0.326316,165.284967,26.394980,0.263069,0.317349
6,10074,0.222886,59.044260,22.497083,0.186718,0.881537
7,11753,0.422842,77.494742,26.082878,0.384982,-0.020745
8,13432,0.297266,132.214791,25.065246,0.270560,0.722780
9,15111,0.156281,31.975990,15.485721,0.117847,0.937389


#### Mean Statistics PCA-NSFTS:

In [28]:
final_result = {
     "rmse": [],
     "mape": [],
     "smape": [],
     "mae": [],
     "r2": []
}

final_result["rmse"].append(statistics.mean(measures['rmse']))
final_result["mape"].append(statistics.mean(measures['mape']))
final_result["smape"].append(statistics.mean(measures['smape']))
final_result["mae"].append(statistics.mean(measures['mae']))
final_result["r2"].append(statistics.mean(measures['r2']))
        
final_measures_pca = pd.DataFrame(final_result)

print("Mean Statistics PCA-NSFTS (test): ")
final_measures_pca

Mean Statistics PCA-NSFTS (test): 


,rmse,mape,smape,mae,r2
0,0.316,102.477686,19.311284,0.219181,0.744296


### Sliding Window PyFTS: KPCA-NSFTS

In [29]:
result = {
     "window": [],
     "rmse": [],
     "mape": [],
     "smape": [],
     "mae": [],
     "r2": []
}

tam = len(data)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
    if len(ttest) > 0:
        
        print('-' * 20)
        print(f'training window {(ct)}')
        ttrain = ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
        model_train, pca_train = fts_pca.run_train_kernel_pca_non_stationary(ttrain)
        y_test = ttest['use [kW](t-1)'].values
        forecast, data_test = fts_pca.run_test_target(y_test,steps_ahead)
        y_validation = ttest['use [kW](t)'].values
        
        y_validation = y_validation[:len(y_validation)-1]
        forecast = forecast[1:]
        
        #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        rmse = Measures.rmse(y_validation,forecast)
        mape = Measures.mape(y_validation,forecast)
        smape = Measures.smape(y_validation,forecast)
        mae = mean_absolute_error(y_validation, forecast)
        r2 = r2_score(y_validation, forecast)
        
        result["rmse"].append(rmse)
        result["mape"].append(mape)
        result["smape"].append(smape)
        result["mae"].append(mae)
        result["r2"].append(r2)
        result["window"].append(ct)
        
measures = pd.DataFrame(result)


--------------------
training window 0
--------------------
training window 1679
--------------------
training window 3358
--------------------
training window 5037
--------------------
training window 6716
--------------------
training window 8395
--------------------
training window 10074
--------------------
training window 11753
--------------------
training window 13432
--------------------
training window 15111
--------------------
training window 16790
--------------------
training window 18469
--------------------
training window 20148
--------------------
training window 21827
--------------------
training window 23506
--------------------
training window 25185
--------------------
training window 26864
--------------------
training window 28543
--------------------
training window 30222
--------------------
training window 31901
--------------------
training window 33580
--------------------
training window 35259
--------------------
training window 36938
--------------------

#### Sliding Windows Statistics: 

In [30]:
#print("Sliding Windows Statistics: ")
measures

,window,rmse,mape,smape,mae,r2
0,0,0.279610,27.017090,15.933216,0.224478,0.848462
1,1679,0.175538,17.073140,8.210064,0.104695,0.931558
2,3358,0.269834,45.622142,20.939252,0.174740,0.624593
3,5037,0.264021,19.028480,9.703751,0.177589,0.874372
4,6716,0.163490,41.843401,11.521080,0.104617,0.919866
5,8395,0.207529,43.953692,22.914382,0.140075,0.723892
6,10074,0.222777,57.498940,14.704066,0.158205,0.881653
7,11753,0.283440,31.487408,19.914578,0.187447,0.541348
8,13432,0.288619,53.607801,22.095015,0.170935,0.738673
9,15111,0.234265,33.666659,20.120704,0.157368,0.859311


#### Mean Statistics KPCA-NSFTS:

In [31]:
final_result = {
     "rmse": [],
     "mape": [],
     "smape": [],
     "mae": [],
     "r2": []
}

final_result["rmse"].append(statistics.mean(measures['rmse']))
final_result["mape"].append(statistics.mean(measures['mape']))
final_result["smape"].append(statistics.mean(measures['smape']))
final_result["mae"].append(statistics.mean(measures['mae']))
final_result["r2"].append(statistics.mean(measures['r2']))
        
final_measures_kpca = pd.DataFrame(final_result)

print("Mean Statistics KPCA-NSFTS: ")
final_measures_kpca

Mean Statistics KPCA-NSFTS: 


,rmse,mape,smape,mae,r2
0,0.342898,55.0952,17.798636,0.181442,0.769116
